In [ ]:
from high_low_xuejieZuhui import *
from horizontal_area import *
import os
from test import *


os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606')
for filename in os.listdir():
    # filename = '000001.SZ.csv'
    os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606/_horizontal_area_pics')
    df = df_init(filename)
    peaks, valleys, high_points, low_points = find_high_low_xuejieZuhui(
        df, filename, save_data=False, draw_n_days=200, draw=False)

    max_len_of_window = 40
    min_len_of_window = 12
    gamma = 0.8  # price_change_50 >= gamma * max_change:
    view_coe = 0.8  # 视野·系数

    result = find_horizontal_area1(df, high_points, low_points, max_len_of_window,
                                min_len_of_window, gamma, view_coe, ignore_hl=True)

    n_days = 300
    draw_horizontal_area(df, result, peaks, valleys, high_points,
                        low_points, filename, n_days, print_result=False, show_plot=False)
                        
    os.chdir('/Users/kai/Desktop/qs/data_csv_distinct_0606')


In [ ]:
from high_low_xuejieZuhui import *
from horizontal_area import *
import os
from test import *

In [ ]:
# filename = '688699.SH.csv'
filename = '000001.SZ.csv'
df = df_init(filename)
peaks, valleys, high_points, low_points = find_high_low_xuejieZuhui(
    df, filename, save_data=False, draw_n_days=200, draw=False)

max_len_of_window = 40
min_len_of_window = 10
gamma = 0.7  # price_change_50 >= gamma * max_change:
view_coe = 0  # 视野·系数
var = 30

result = find_horizontal_area1(df, high_points, low_points, var, max_len_of_window,
                            min_len_of_window, gamma, view_coe, ignore_hl=True)

n_days = 1000
draw_horizontal_area(df, result, peaks, valleys, high_points,
                    low_points, filename, n_days, print_result=True, show_plot=True)

In [ ]:
n_days = 400
draw_horizontal_area(df, result, peaks, valleys, high_points,
                    low_points, filename, n_days, print_result=False, show_plot=True)

In [2]:
for i in [1,2,30]:
    print(i)

1
2
30
